In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Standard libraries
import logging
import os
import pathlib
import sys

# 3rd party libraries
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
import sqlalchemy as sa

# Local libraries
import pudl
import pudl.output.ferc714

# Configure Display Parameters

In [ ]:
sns.set()
%matplotlib inline
mpl.rcParams['figure.figsize'] = (10,4)
mpl.rcParams['figure.dpi'] = 150
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

# Use Python Logging facilities
* Using a logger from the beginning will make the transition into the PUDL package easier.
* Creating a logging handler here will also allow you to see the logging output coming from PUDL and other underlying packages.

In [ ]:
logger=logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

# Define Functions

# Define Notebook Parameters

In [ ]:
from pudl.workspace.setup import PudlPaths

ferc1_engine = sa.create_engine(PudlPaths().sqlite_db_uri("ferc1"))
display(ferc1_engine)

pudl_engine = sa.create_engine(PudlPaths().pudl_db)
display(pudl_engine)

# Load Data

In [ ]:
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine=pudl_engine)

In [ ]:
%%time
ferc714_out = pudl.output.ferc714.Respondents(pudl_out)
annualized = ferc714_out.annualize()
categorized = ferc714_out.categorize()
summarized = ferc714_out.summarize_demand()
fipsified = ferc714_out.fipsify()
counties_gdf = ferc714_out.georef_counties()

In [ ]:
categorized.info()

In [ ]:
summarized.info()

In [ ]:
fipsified.info()

In [ ]:
counties_gdf.info()

In [ ]:
# This takes 45 minutes so...
#respondents_gdf = ferc714_out.georef_respondents()
#display(respondents_gdf.info())
#respondents_gdf.sample(10)